<a href="https://colab.research.google.com/github/maimunaBCS16/MlPro/blob/main/cancerB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

start


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#check
cud_d=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(cud_d)

cuda


In [3]:
#loading data
p_data=load_breast_cancer()
x_data=p_data.data
y_data=p_data.target
print(x_data)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


split

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
print(x_data.shape)
print(x_train.shape)
print(x_test.shape)


(569, 30)
(455, 30)
(114, 30)


In [5]:
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)
type(x_train)


numpy.ndarray

data>>pyTorch

In [6]:
x_train=torch.tensor(x_train, dtype=torch.float32).to(cud_d)
x_test=torch.tensor(x_test, dtype=torch.float32).to(cud_d)
y_train=torch.tensor(y_train, dtype=torch.float32).to(cud_d)
y_test=torch.tensor(y_test, dtype=torch.float32).to(cud_d)


In [7]:
#nural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [8]:
input_size = x_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100
model=NeuralNet(input_size, hidden_size, output_size).to(cud_d)

In [9]:
criterion=nn.BCELoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

train

In [27]:
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs=model(x_train)
    loss=criterion(outputs, y_train.view(-1, 1))
    loss.backward()
    optimizer.step()

    with torch.no_grad():
      predicted=outputs.round()
      correct=(predicted==y_train.view(-1, 1)).float().sum()
      accuracy=correct/y_train.size(0)
      if (epoch+1)%10==0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item()*100:.2f}%')


Epoch [10/100], Loss: 0.1112, Accuracy: 97.36%
Epoch [20/100], Loss: 0.1028, Accuracy: 97.80%
Epoch [30/100], Loss: 0.0958, Accuracy: 98.02%
Epoch [40/100], Loss: 0.0899, Accuracy: 98.24%
Epoch [50/100], Loss: 0.0847, Accuracy: 98.46%
Epoch [60/100], Loss: 0.0803, Accuracy: 98.46%
Epoch [70/100], Loss: 0.0763, Accuracy: 98.46%
Epoch [80/100], Loss: 0.0728, Accuracy: 98.46%
Epoch [90/100], Loss: 0.0697, Accuracy: 98.46%
Epoch [100/100], Loss: 0.0668, Accuracy: 98.24%


In [28]:
model.eval()
with torch.no_grad():
  outputs=model(x_train)
  predicted=outputs.round()
  correct=(predicted==y_train.view(-1, 1)).float().sum()
  accuracy=correct/y_train.size(0)
  print(f'Accuracy: {accuracy.item()*100:.2f}%')

Accuracy: 98.24%
